In [9]:
from simulation import *
from power_typeIerror import *
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
readRDS = robjects.r['readRDS']
import numpy as np
import matplotlib.pyplot as plt

In [12]:
def get_pvalue_all_genes(input_data):
    '''get p values of fisher's exact test of all genes from pkl input'''
    dat = load_data(input_data)
    sample_genes = get_gene_table(dat)
    pvalue_all_genes = test_contingency_table(sample_genes)
    return pvalue_all_genes

In [13]:
pvalue = get_pvalue_all_genes("data/del_sim_shape9_scale1_p0.01_N4000_M100_12.data.pkl")

In [14]:
print (len([x for x in pvalue["p_value"] if x < 0.05]))
# There are 564 genes whose fisher exact pvalue less than 0.05

943


In [15]:
plt.scatter(range(len(pvalue["p_value"])), sorted(pvalue["p_value"]))
plt.ylabel("pvalue of fisher exact test")
# plt.show()
plt.savefig("pvalue_dist.png")
plt.close()

In [2]:
def get_lfdr_all_genes(rds_data):
    df = readRDS(rds_data)
    df = pandas2ri.ri2py(df)
    pip_rds = df[20]
    # pip_genes = list(pip_rds.names[0])
    pip_list = np.array(np.matrix(pip_rds)).reshape(-1,).tolist()
    lfdr = [(1-x) for x in pip_list]
    return lfdr

In [3]:
lfdr = get_lfdr_all_genes("data/del_sim_shape9_scale1_p0.01_N4000_M100_12.data.feather.RDS")

In [7]:
plt.scatter(range(len(lfdr)), sorted(lfdr))
plt.ylabel("local false discovery rate")
# plt.show()
plt.savefig("lfdr_dist.png")
plt.close()

In [31]:
plt.hist(pvalue["p_value"])
# plt.show()
plt.savefig("pvalue_hist")
plt.close()

In [32]:
plt.hist(lfdr)
plt.savefig("lfdr_hist")
plt.close()

In [58]:
len([x for x in lfdr if x < 0.05])

10

In [38]:
fdr = robjects.r('''
library("fdrtool")
f <- function(lfdr){
    fdr = fdrtool(lfdr, statistic = "pvalue")
    return (fdr$lfdr)
}
''')

In [40]:
r_f = robjects.r["f"]

In [47]:
r_f(0.2)

Step 1... determine cutoff point

Step 2... estimate parameters of null distribution and eta0

Step 3... compute p-values and estimate empirical PDF/CDF

Step 4... compute q-values and local fdr

Step 5... prepare for plotting





R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x7fa906dc4508 / R:0x70e47c8>
[1.000000]

In [50]:
print (len(pvalue["p_value"]))

22159
